---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def hist_generator(G):
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
        return histogram
    

def graph_identification():
        list_outcomes = []
        for G in P1_Graphs:
            clustering = nx.average_clustering(G)
            shortest_path = nx.average_shortest_path_length(G)
            degree_hist = hist_generator(G)
            if len(degree_hist)>10:
                list_outcomes.append('PA')
            elif clustering < 0.1:
                list_outcomes.append('SW_H')
            else:
                list_outcomes.append('SW_L')
        return list_outcomes
    
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [11]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
    
def salary_predictions():
    
    def check_manager_level(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None
    
    df = pd.DataFrame(index=G.nodes())
    df['pr'] = pd.Series(nx.pagerank(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['degree'] = pd.Series(G.degree())
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['is_management'] = pd.Series([check_manager_level(node) for node in G.nodes(data=True)])
    
    df_train = df[~pd.isnull(df['is_management'])]
    df_test = df[pd.isnull(df['is_management'])]
    
    ft_selection = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    X_train = df_train[ft_selection]
    Y_train = df_train['is_management']
    X_test = df_test[ft_selection]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = SVC(probability=True)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)
    
salary_predictions()

1       0.133786
2       0.688356
5       0.999986
8       0.103432
14      0.401735
18      0.196161
27      0.272428
30      0.316711
31      0.147770
34      0.114657
37      0.088355
40      0.205909
45      0.146977
54      0.204403
55      0.189301
60      0.181057
62      0.999999
65      0.720226
77      0.061476
79      0.108151
97      0.079141
101     0.055075
103     0.181308
108     0.107856
113     0.518800
122     0.048023
141     0.491903
142     0.948127
144     0.054780
145     0.394277
          ...   
913     0.046557
914     0.065322
915     0.006783
918     0.110269
923     0.045123
926     0.073793
931     0.049904
934     0.016663
939     0.015310
944     0.006980
945     0.044185
947     0.156723
950     0.092687
951     0.042701
953     0.029406
959     0.007359
962     0.007451
963     0.113357
968     0.125747
969     0.117714
974     0.079434
984     0.015759
987     0.091996
989     0.126464
991     0.126270
992     0.008689
994     0.007444
996     0.0070

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

def new_connections_predictions():
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df = future_connections.join(df,how='outer')
    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    ft_selection = ['cn_soundarajan_hopcroft', 'preferential_attachment']
    X_train = df_train[ft_selection]
    Y_train = df_train['Future Connection']
    X_test = df_test[ft_selection]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = LogisticRegression()
    clf.fit(X_train_scaled, Y_train)
    
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']
    
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.037633
(542, 751)    0.015838
(20, 426)     0.567526
(50, 989)     0.015904
(942, 986)    0.015975
(324, 857)    0.015895
(13, 710)     0.174663
(19, 271)     0.160110
(319, 878)    0.015927
(659, 707)    0.015845
(49, 843)     0.015947
(208, 893)    0.015861
(377, 469)    0.013232
(405, 999)    0.025141
(129, 740)    0.024212
(292, 618)    0.024360
(239, 689)    0.015931
(359, 373)    0.014292
(53, 523)     0.091979
(276, 984)    0.015944
(202, 997)    0.015958
(604, 619)    0.039848
(270, 911)    0.015930
(261, 481)    0.083063
(200, 450)    0.984811
(213, 634)    0.015818
(644, 735)    0.039695
(346, 553)    0.015654
(521, 738)    0.015337
(422, 953)    0.024103
                ...   
(672, 848)    0.015930
(28, 127)     0.906042
(202, 661)    0.015626
(54, 195)     0.998621
(295, 864)    0.015885
(814, 936)    0.015828
(839, 874)    0.015975
(139, 843)    0.015891
(461, 544)    0.015116
(68, 487)     0.015077
(622, 932)    0.015869
(504, 936)    0.023317
(479, 528) 